In [4]:
import os

import psycopg 

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

from dotenv import load_dotenv, find_dotenv

TABLE_NAME = "users_churn" 

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "transformers_creation"
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "transformers_model"

In [5]:
# подгружаем .env
load_dotenv()

True

In [6]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [7]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,6023,5025-GOOKI,2018-06-01,NaT,Month-to-month,Yes,Credit card (automatic),18.9,347.65,None,...,None,None,None,None,Female,0,No,No,No,0
1,6024,4698-KVLLG,2015-11-01,NaT,Two year,No,Credit card (automatic),19.6,967.90,None,...,None,None,None,None,Female,1,No,No,No,0


In [5]:
### ЗАДАНИЕ 1

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# Создание объекта OneHotEncoder для преобразования категориальных переменных
encoder_oh = OneHotEncoder(
    categories="auto",  # Автоматическое определение категорий
    handle_unknown="ignore",  # Игнорировать неизвестные категории
    max_categories=10,  # Максимальное количество уникальных категорий
    sparse_output=False,  # Вывод в виде обычного массива
    drop="first"  # Удаляет первую категорию, чтобы избежать мультиколлинеарности
)

# Применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

# Преобразование полученных признаков в DataFrame и установка названий колонок
encoded_df = pd.DataFrame(
    encoded_features,
    columns=encoder_oh.get_feature_names_out(cat_columns)  # Получение имён признаков после преобразования
)

# Конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = df.select_dtypes(include="object")
obj_df = pd.concat([obj_df, encoded_df], axis=1)

In [12]:
### ЗАДАНИЕ 2

In [9]:
df[["monthly_charges", "total_charges"]]

,monthly_charges,total_charges
0,18.90,347.65
1,19.60,967.90
2,91.50,242.95
3,45.20,1841.90
4,19.45,232.10
...,...,...
7038,111.95,8033.10
7039,19.85,1253.65
7040,89.75,552.65
7041,20.05,1036.00


In [10]:
import pandas as pd
from sklearn.preprocessing import SplineTransformer, QuantileTransformer, RobustScaler, PolynomialFeatures, KBinsDiscretizer

# Исходные данные
num_columns = ["monthly_charges", "total_charges"]

# Параметры
n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# Удалим строки с нанами в num_columns (SplineTransformer падает с ошибкой)
df = df[~(df["monthly_charges"].isnull() | df["total_charges"].isnull())].reset_index(drop=True)

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([df[num_columns], encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

In [9]:
### ЗАДАНИЕ 3

In [11]:
# Преобразования для числовых колонок
numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns), 
                                                      ('q', encoder_q, num_columns), 
                                                      ('rb', encoder_rb, num_columns), 
                                                      ('pol', encoder_pol, num_columns), 
('kbd', encoder_kbd, num_columns)])


# Преобразования для категориальных колонок
categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

# Объединение всех преобразований
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)

# Применение преобразований
encoded_features = preprocessor.fit_transform(df)

# Преобразование результата в DataFrame
encoded_columns = (
    preprocessor.named_transformers_["num"].get_feature_names_out(num_columns).tolist()
    + preprocessor.named_transformers_["cat"].get_feature_names_out(cat_columns).tolist()
)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

# Объединение с исходным DataFrame
df = pd.concat([df, transformed_df], axis=1)

# Вывод результата
print(df.head(2))

     id customer_id begin_date end_date            type paperless_billing   
0  6023  5025-GOOKI 2018-06-01      NaT  Month-to-month               Yes  \
1  6024  4698-KVLLG 2015-11-01      NaT        Two year                No   

            payment_method  monthly_charges  total_charges internet_service   
0  Credit card (automatic)             18.9         347.65             None  \
1  Credit card (automatic)             19.6         967.90             None   

   ... num__kbd__monthly_charges num__kbd__total_charges cat__type_One year   
0  ...                       0.0                     0.0                0.0  \
1  ...                       0.0                     0.0                0.0   

  cat__type_Two year cat__payment_method_Credit card (automatic)   
0                0.0                                         1.0  \
1                1.0                                         1.0   

  cat__payment_method_Electronic check cat__payment_method_Mailed check   
0           

In [31]:
### ЗАДАНИЕ 4

In [34]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/03/18 20:49:47 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [ ]:
### ЗАДАНИЕ 5

In [11]:
# Укажите путь к файлу
import joblib

# Загружаем модель
with open('../churn_model/fitted_model.pkl', 'rb') as fd:
    fit_pipeline = joblib.load(fd)

In [13]:
base_features = ['end_date', 'begin_date', 'type',
                 'total_charges', 'multiple_lines', 'gender', 'streaming_tv', 'payment_method', 'device_protection', 
                 'tech_support', 'online_security', 'internet_service', 'partner', 'monthly_charges', 
                 'online_backup', 'streaming_movies', 'dependents', 'paperless_billing']

#'total_charges' 'senior_citizen'
feature_engineering = ['num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__internet_service_None', 'cat__gender_Male']

In [22]:
df[base_features]

,end_date,begin_date,type,total_charges,multiple_lines,gender,streaming_tv,payment_method,device_protection,tech_support,online_security,internet_service,partner,monthly_charges,online_backup,streaming_movies,dependents,paperless_billing
0,NaT,2018-06-01,Month-to-month,347.65,No,Female,None,Credit card (automatic),None,None,None,None,No,18.90,None,None,No,Yes
1,NaT,2015-11-01,Two year,967.90,No,Female,None,Credit card (automatic),None,None,None,None,No,19.60,None,None,No,No
2,2019-12-01,2019-09-01,Month-to-month,242.95,Yes,Female,No,Credit card (automatic),No,No,No,Fiber optic,Yes,91.50,Yes,Yes,No,Yes
3,NaT,2016-09-01,Month-to-month,1841.90,No,Female,No,Bank transfer (automatic),No,No,No,DSL,No,45.20,No,No,Yes,No
4,NaT,2019-01-01,Month-to-month,232.10,No,Female,None,Credit card (automatic),None,None,None,None,No,19.45,None,None,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,NaT,2014-02-01,Two year,8033.10,No,Male,Yes,Bank transfer (automatic),Yes,Yes,Yes,Fiber optic,Yes,111.95,Yes,Yes,Yes,No
7028,NaT,2014-12-01,Two year,1253.65,No,Male,None,Bank transfer (automatic),None,None,None,None,No,19.85,None,None,No,No
7029,NaT,2019-08-01,Month-to-month,552.65,Yes,Female,Yes,Electronic check,No,No,No,Fiber optic,No,89.75,Yes,No,No,Yes
7030,NaT,2016-02-01,Two year,1036.00,No,Male,None,Mailed check,None,None,None,None,Yes,20.05,None,None,No,Yes


In [1]:
# transformed_data = fit_pipeline.named_steps['preprocessor'].transform(df[base_features])

In [28]:
# Соберем некоторый набор признаков

features_list = ['type',
       'paperless_billing', 'payment_method', 'monthly_charges',
       'total_charges', 'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines',
       'num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__internet_service_None', 'cat__gender_Male']

In [16]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [14]:
X = df[feature_engineering]  # Признаки
y = df['target']  # Целевая переменная

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [15]:
model_new = CatBoostClassifier(depth=5, iterations=1001)
model_new.fit(X_train, y_train)

Learning rate set to 0.020327
0:	learn: 0.6831988	total: 59.9ms	remaining: 59.9s
1:	learn: 0.6714758	total: 64.4ms	remaining: 32.2s
2:	learn: 0.6601373	total: 68.8ms	remaining: 22.9s
3:	learn: 0.6505827	total: 73.2ms	remaining: 18.2s
4:	learn: 0.6419386	total: 77.5ms	remaining: 15.4s
5:	learn: 0.6318872	total: 81.8ms	remaining: 13.6s
6:	learn: 0.6226624	total: 87.4ms	remaining: 12.4s
7:	learn: 0.6156109	total: 91.7ms	remaining: 11.4s
8:	learn: 0.6080947	total: 96.1ms	remaining: 10.6s
9:	learn: 0.5997408	total: 101ms	remaining: 9.98s
10:	learn: 0.5919900	total: 105ms	remaining: 9.46s
11:	learn: 0.5851584	total: 110ms	remaining: 9.03s
12:	learn: 0.5785872	total: 114ms	remaining: 8.66s
13:	learn: 0.5724040	total: 118ms	remaining: 8.32s
14:	learn: 0.5661339	total: 122ms	remaining: 8.04s
15:	learn: 0.5590996	total: 126ms	remaining: 7.78s
16:	learn: 0.5543783	total: 131ms	remaining: 7.58s
17:	learn: 0.5499862	total: 135ms	remaining: 7.38s
18:	learn: 0.5451155	total: 140ms	remaining: 7.21s
19

In [ ]:
import mlflow
import numpy as np

# Настройка подключения к tracking и registry
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')
mlflow.set_registry_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')

EXPERIMENT_NAME = "churn_mmakarov_fe_model"
RUN_NAME = "m1"
REGISTRY_MODEL_NAME = "churn_model_FE_data"

# Создаём или получаем эксперимент
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id


# Путь до файла с зависимостями pip
pip_requirements = "../requirements.txt"

# Формирование сигнатуры модели (входы и выходы)
signature = mlflow.models.infer_signature(
    df.values,
    model_new.predict_proba(X_test)[:, 1]
)

# Пример входных данных для модели
input_example = [df.values[0], df.values[1]] 

# Пути до скрипта с обучением модели
code_paths = ["../churn_model/fit.py"]

# Предположим, что модель уже обучена или инициализирована (в данном примере используем модель CatBoost)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    # Логирование модели с передачей всех необходимых параметров и регистрация в реестре
    # model_info = mlflow.pyfunc.log_model(
    #     artifact_path="models",  # Путь для артефактов в реестре
    #     cb_model=model,  # Передаем модель CatBoost с параметром cb_model
    #     pip_requirements=pip_requirements,
    #     signature=signature,
    #     input_example=input_example,
    #     code_paths=code_paths,
    #     registered_model_name=REGISTRY_MODEL_NAME
    # )

    # Логируем метрики (например, точность на обучающих данных)
    roc_auc = roc_auc_score(y_test, model_new .predict_proba(X_test)[:, 1])
    precision = precision_score(y_test, model_new .predict(X_test))
    recall = recall_score(y_test, model_new.predict(X_test))
    f1 = f1_score(y_test, model_new.predict(X_test))

    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)


    model_info = mlflow.catboost.log_model(
        cb_model=model_new,  # Передаем модель CatBoost
        artifact_path="models",  # Путь для артефактов в реестре
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        registered_model_name=REGISTRY_MODEL_NAME
    )

    print(f"Model registered at: {model_info.model_uri}")